# Cat vs. Dog Classifier

In [1]:
import pdb
import numpy as np

import tensorflow as tf

import keras
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2D, Activation
from keras.preprocessing import image

# DO I NEED IT?
# import sys
# from PIL import Image
# sys.modules['Image'] = Image 

Using TensorFlow backend.


## Neural Network
In order 

## Create additional data

* jaki mam problem - mało danych
* po co to robię 
* co to robi

In [2]:
train_datagen = image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = image.ImageDataGenerator(rescale=1./255)

In [3]:
train_set_path = 'dataset/training_set'
test_set_path = 'dataset/test_set'

train_set = train_datagen.flow_from_directory(
        train_set_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        test_set_path,
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


# Neural Network

Neural Network that will be used for this task is a Convolutional Neural Network.
It consists of the follwing layers: 
* 
* 
* 

In [4]:
def create_model(num_classes=2):

    model = Sequential()

    model.add(Convolution2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Convolution2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
#     model.add(Convolution2D(128, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


model = create_model()

W0703 11:50:09.872538 139977731734400 deprecation_wrapper.py:119] From /home/pietrek/ML/CNN/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0703 11:50:09.887133 139977731734400 deprecation_wrapper.py:119] From /home/pietrek/ML/CNN/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0703 11:50:09.889093 139977731734400 deprecation_wrapper.py:119] From /home/pietrek/ML/CNN/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0703 11:50:09.901951 139977731734400 deprecation_wrapper.py:119] From /home/pietrek/ML/CNN/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d inste

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total para

### Keras Callbacks
Since training of the neural network takes a really long time, it is a good idea to save the progress of this process.   
Fortunately Keras has an adequate feature - callbacks. This requires only creating of the callback object that will contain an adequate method - in this case `on_epoch_end` due to the fact, that the progress will be saved each time an epoch ends.

In [6]:
class ModelSaver(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.save(f'model_{epoch}.hd5')

### Train the network...

Side note - unfortunately the version of Keras (2.2.4) that was used to build the network had problems with using `validation_data` parameter in `fit_generator` function. Training process stopped every time on the last entity of the first epoch, which made training impossible.   
In order to train the network said parameter was erased - network was tested later.   

In [7]:
# model.fit_generator(
#         train_set,
#         steps_per_epoch=10,
#         epochs=20,
#     callbacks=[ModelSaver()],
# #         validation_data=test_set,
# #         validation_steps=2000
# )

### ... or load it

In [8]:
model = load_model('model.h5')

W0703 11:50:10.110922 139977731734400 deprecation_wrapper.py:119] From /home/pietrek/ML/CNN/venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



## Visualise history 
https://www.kaggle.com/uysimty/keras-cnn-dog-or-cat-classification

# Perform check 


In [71]:
import os
image_path = os.getcwd() + f'{os.sep}dataset{os.sep}test_set{os.sep}'
cats_path = image_path + f'cats{os.sep}'
dogs_path = image_path + f'dogs{os.sep}'

def load_image(image_path):
    test_image = image.load_img(image_path, target_size = (64,64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    return test_image

def create_set(path, label):
    files = os.listdir(path)
    paths = [path + file for file in files if file != '.DS_Store']
    no_of_images = len(paths)
    images = []
    for path in paths: 
        images.append(load_image(path))
    return images, no_of_images * [label]
  
cats_images, cat_labels = create_set(cats_path, '0')
dogs_images, dog_labels = create_set(dogs_path, '0')


X_test = cats_images + dogs_images 
y_test = cat_labels + dog_labels

In [11]:
# t = [test_example for test_example in test_set]

count = 0
test = []

for t in test_set:
        
    if count == 10:
        break
    count = count + 1
    test.append(t)

In [49]:
X_test = []
y_test = []

for el in test: 
    X_test.append(el[0])
    y_test.append(el[1])
    
    
X_test = np.array(X_test)
# y_test = np.array(y_test)

def flatten(l): 
    return [item for sublist in l for item in sublist]


def reshape_list(l):
    return l.reshape(-1, l.shape[-1])

# X_test = reshape_list(X_test)
# a = test[0][0]
# b = test[0][1]

# a = flatten(a)
# b = flatten(b)

# model.evaluate(a, b)
X_test = X_test.reshape([-1,32, 64, 64, 3])
# y_test = y_test.reshape([-1,32, 64, 64, 3])
X_test
# model.evaluate(X_test, y_test)

# # [0]
# X_test = np.array(X_test)


array([[[[[0.4784314 , 0.52156866, 0.38823533],
          [0.5568628 , 0.57254905, 0.4431373 ],
          [0.5803922 , 0.56078434, 0.4431373 ],
          ...,
          [0.6862745 , 0.70980394, 0.5294118 ],
          [0.69411767, 0.6862745 , 0.53333336],
          [0.7019608 , 0.6901961 , 0.5294118 ]],

         [[0.5294118 , 0.5882353 , 0.41960788],
          [0.5411765 , 0.5882353 , 0.43921572],
          [0.5882353 , 0.6431373 , 0.5058824 ],
          ...,
          [0.6509804 , 0.5058824 , 0.43529415],
          [0.69803923, 0.654902  , 0.54509807],
          [0.58431375, 0.5529412 , 0.46274513]],

         [[0.40000004, 0.46274513, 0.31764707],
          [0.5137255 , 0.6039216 , 0.49411768],
          [0.42352945, 0.5019608 , 0.3647059 ],
          ...,
          [0.7176471 , 0.6039216 , 0.57254905],
          [0.7725491 , 0.6117647 , 0.59607846],
          [0.6745098 , 0.57254905, 0.4666667 ]],

         ...,

         [[0.09803922, 0.18039216, 0.12156864],
          [0.3254902 ,

# Prediction on own images
From this point it is fairly easy to create a program that will perform a prediciton on an images provided by user. First step to do that is to save the model - this will allow to get the response immediately, instead of train the classifier each time.

In [69]:
model.evaluate(new_arr, np.array(y_test))

ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (8192000, 3)

## 1) Save the model
Saving the model using Keras is really straightforward - it requires running `save()` method on the model's object with model's path as its parameter.

In [ ]:
model.save('model.h5')

## 2) Get and convert images

Section below contains a code that performs some manipulations on an image in order to be able to perform a prediction on it.   

The manipulations work as follows:   
1. Load an image from the disk and resize it to 64x64 pixels.  
2. Convert it to an array.  
3. Add additional dimension - it is required since the neural network requires a tensor containing four dimensions.  


In [ ]:
image_path = 'dataset/single_prediction/cat.jpg'

test_image = image.load_img(image_path, target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

## 3) Run the prediction
Once an image is loaded and converted to an adequate representation, it is possible to perform a prediction. In order to do that method `predict()` is run.
It returns a list of ...

In [59]:
result = model.predict(test_image)[0][0]
if result == 1:
    print('It is a dog!')
else: 
    print('It is a cat!')

It is a dog!


In [58]:
test_image = image.load_img('dataset/single_prediction/corgi.jpg',
                           target_size = (64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
Y = model.predict_classes(test_image)
Y

array([[1]], dtype=int32)

## 4) Standalone program
Feel free to check program `animal_predictor.py` that is located in the same directory as this notebook.  It is a simple command line application that contains the whole functionality of predicting the animal on the provided image.

### 4a) Instruction
To perform a prediction `animal_predictior.py` requires to arguments:
* path to the model
* path to the image

Example usage: `python animal_predictor.py --image 'dataset/dog_image.jpg' --model 'model.h5'`